In [15]:
from fairness.matching import compute
import pandas as pd
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import accuracy_score,confusion_matrix
from sklearn.metrics import classification_report
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import ConfusionMatrixDisplay
from sklearn.model_selection import GridSearchCV
import xgboost as xgb
import matplotlib.pyplot as plt

In [16]:
dataset = pd.read_csv('./dataset/student-por.csv', sep=';')

In [17]:
dataset

,school,sex,age,address,famsize,Pstatus,Medu,Fedu,Mjob,Fjob,...,famrel,freetime,goout,Dalc,Walc,health,absences,G1,G2,G3
0,GP,F,18,U,GT3,A,4,4,at_home,teacher,...,4,3,4,1,1,3,4,0,11,11
1,GP,F,17,U,GT3,T,1,1,at_home,other,...,5,3,3,1,1,3,2,9,11,11
2,GP,F,15,U,LE3,T,1,1,at_home,other,...,4,3,2,2,3,3,6,12,13,12
3,GP,F,15,U,GT3,T,4,2,health,services,...,3,2,2,1,1,5,0,14,14,14
4,GP,F,16,U,GT3,T,3,3,other,other,...,4,3,2,1,2,5,0,11,13,13
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
644,MS,F,19,R,GT3,T,2,3,services,other,...,5,4,2,1,2,5,4,10,11,10
645,MS,F,18,U,LE3,T,3,1,teacher,services,...,4,3,4,1,1,1,4,15,15,16
646,MS,F,18,U,GT3,T,1,1,other,other,...,1,1,1,1,1,5,6,11,12,9
647,MS,M,17,U,LE3,T,3,1,services,services,...,2,4,5,3,4,2,6,10,10,10


In [18]:
columns_to_drop = ['school', 'address', 'G1', 'G2']
protected_attributes = ['sex', 'health', 'romantic']
output_column = 'G3'

In [19]:
dataset.columns

Index(['school', 'sex', 'age', 'address', 'famsize', 'Pstatus', 'Medu', 'Fedu',
       'Mjob', 'Fjob', 'reason', 'guardian', 'traveltime', 'studytime',
       'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'nursery',
       'higher', 'internet', 'romantic', 'famrel', 'freetime', 'goout', 'Dalc',
       'Walc', 'health', 'absences', 'G1', 'G2', 'G3'],
      dtype='object')

In [20]:
fair_dataset = compute(dataset, protected_attributes, output_column, columns_to_drop)

unfair


KeyboardInterrupt: 

In [6]:
fair_dataset

,1,2,3,4,5,6,7,8,9,10,...,22,23,24,25,26,27,28,29,30,GRADE
0,1,2,3,3,1,2,2,0,0,1,...,1,1,1,3,2,1,2,1,1,1
1,1,2,3,3,1,2,2,0,0,1,...,1,1,1,3,2,3,2,2,3,1
2,1,2,2,3,2,2,2,1,1,2,...,1,1,1,2,2,1,1,2,2,1
3,0,1,1,3,1,2,1,1,0,2,...,1,1,2,3,2,2,1,3,2,1
4,1,2,1,3,2,2,1,1,0,4,...,1,2,1,2,2,2,1,2,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
140,1,1,2,3,1,1,2,0,0,2,...,1,1,1,2,1,2,1,3,3,5
141,0,1,2,4,2,2,2,0,1,2,...,2,1,1,3,2,2,1,5,3,5
142,0,1,1,4,2,2,2,0,0,1,...,1,1,1,3,3,2,1,4,3,1
143,1,1,2,4,1,1,1,1,1,3,...,1,2,1,2,1,2,1,5,3,4


In [7]:
X = fair_dataset.iloc[:,:len(fair_dataset.columns) - 2]
y = fair_dataset.loc[:, output_column:]

In [8]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [9]:
standard_scaler_train = StandardScaler()
X_train = standard_scaler_train.fit_transform(X_train)

standard_scaler_test = StandardScaler()
X_test = standard_scaler_test.fit_transform(X_test)

In [10]:
estimator = DecisionTreeClassifier()

parameters = {
    'max_depth': [2, 3, 5, 10, 20],
    'min_samples_leaf': [5, 10, 20, 50, 100],
    'criterion': ["gini", "entropy"]
}

In [11]:
grid_search = GridSearchCV(
    estimator=estimator,
    param_grid=parameters,
    n_jobs=10,
    cv=10,
    verbose=True
)

In [12]:
grid_search_classifier = grid_search.fit(X_train, y_train)

Fitting 10 folds for each of 50 candidates, totalling 500 fits


C:\Users\anton\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.10_qbz5n2kfra8p0\LocalCache\local-packages\Python310\site-packages\sklearn\model_selection\_split.py:700: UserWarning: The least populated class in y has only 4 members, which is less than n_splits=10.
  warnings.warn(


In [13]:
y_pred = grid_search_classifier.predict(X_test)

In [14]:
print(accuracy_score(y_test, y_pred))

0.20689655172413793
